## Modele

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

def get_fashion_mnist_dataloaders(
    train_batch_size: int = 64,
    val_batch_size: int = 64,
    val_split: float = 0.2,
    seed: int = 42
):

    torch.manual_seed(seed)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_dataset = datasets.FashionMNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    val_size = int(len(train_dataset) * val_split)
    train_size = len(train_dataset) - val_size

    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(
        train_dataset,
        batch_size=train_batch_size,
        shuffle=True,
        num_workers=2
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=val_batch_size,
        shuffle=False,
        num_workers=2
    )

    return train_loader, val_loader

In [ ]:
from torch import nn
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class SimpleFCN(nn.Module):
    def __init__(self):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def get_mobilenet(num_classes: int = 100, in_chanels = 3) -> nn.Module:

    model_weights = torchvision.models.MobileNet_V3_Small_Weights.DEFAULT
    transforms = model_weights.transforms()
    model = torchvision.models.mobilenet_v3_small(weights=model_weights)

    if in_chanels != 3:
      model.features[0][0] = nn.Conv2d(
          in_channels=in_chanels,
          out_channels=model.features[0][0].out_channels,
          kernel_size=model.features[0][0].kernel_size,
          stride=model.features[0][0].stride,
          padding=model.features[0][0].padding,
          bias=False
      )


    for param in model.features.parameters():
        param.requires_grad = False
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

    return model, transforms

In [ ]:
def get_effnetb0(out_features):
    effnetb0_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    transforms = effnetb0_weights.transforms()
    model = torchvision.models.efficientnet_b0(weights=effnetb0_weights) # noqa 5501

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280,
                  out_features=out_features,
                  bias=True))

    model.name = "effnetb0"
    return model, transforms

In [ ]:
def get_effnetb7(out_features):
    effnetb7_weights = torchvision.models.EfficientNet_B7_Weights.DEFAULT
    transforms = effnetb7_weights.transforms()
    model = torchvision.models.efficientnet_b7(weights=effnetb7_weights) # noqa 5501

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=2560,
                  out_features=out_features,
                  bias=True))

    model.name = "effnetb7"
    return model, transforms

In [ ]:
from torch import optim
from tqdm import tqdm

def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    num_epochs: int = 5,
    learning_rate: float = 1e-3,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        print(f"Epoch [{epoch + 1}/{num_epochs}]")
        print(f"Train Loss: {train_loss / len(train_loader):.4f}")
        print(f"Val Loss: {val_loss / len(val_loader):.4f}")
        print(f"Val Accuracy: {100. * correct / total:.2f}%")
        print("-" * 50)

In [ ]:
from torchvision import datasets, transforms, models
import torchvision


def get_cifar100_dataloaders(
    train_batch_size: int = 64,
    val_batch_size: int = 64,
    val_split: float = 0.2,
    seed: int = 42,
    transform=None
) -> tuple[DataLoader, DataLoader]:

    train_dataset = datasets.CIFAR100(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    val_size = int(len(train_dataset) * val_split)
    train_size = len(train_dataset) - val_size

    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(
        train_dataset,
        batch_size=train_batch_size,
        shuffle=True,
        num_workers=2
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=val_batch_size,
        shuffle=False,
        num_workers=2
    )

    return train_loader, val_loader

## FashionMNIST

In [ ]:
model = SimpleCNN()
train_dataloader, test_dataloader = get_fashion_mnist_dataloaders()
train_model(model, train_dataloader, test_dataloader)

 20%|██        | 1/5 [00:14<00:56, 14.23s/it]

Epoch [1/5]
Train Loss: 0.5103
Val Loss: 0.3645
Val Accuracy: 87.05%
--------------------------------------------------


 40%|████      | 2/5 [00:28<00:43, 14.36s/it]

Epoch [2/5]
Train Loss: 0.3290
Val Loss: 0.3253
Val Accuracy: 88.16%
--------------------------------------------------


 60%|██████    | 3/5 [00:42<00:28, 14.32s/it]

Epoch [3/5]
Train Loss: 0.2800
Val Loss: 0.2845
Val Accuracy: 89.63%
--------------------------------------------------


 80%|████████  | 4/5 [00:57<00:14, 14.59s/it]

Epoch [4/5]
Train Loss: 0.2464
Val Loss: 0.2689
Val Accuracy: 90.31%
--------------------------------------------------


100%|██████████| 5/5 [01:12<00:00, 14.45s/it]

Epoch [5/5]
Train Loss: 0.2240
Val Loss: 0.2456
Val Accuracy: 90.67%
--------------------------------------------------


In [ ]:
model = get_mobilenet(num_classes=10, in_chanels=1)[0]
train_dataloader, test_dataloader = get_fashion_mnist_dataloaders()
train_model(model, train_dataloader, test_dataloader)

 20%|██        | 1/5 [00:20<01:21, 20.40s/it]

Epoch [1/5]
Train Loss: 1.3990
Val Loss: 1.1597
Val Accuracy: 57.77%
--------------------------------------------------


 40%|████      | 2/5 [00:41<01:02, 20.93s/it]

Epoch [2/5]
Train Loss: 1.2035
Val Loss: 1.1105
Val Accuracy: 59.48%
--------------------------------------------------


 60%|██████    | 3/5 [01:02<00:41, 20.81s/it]

Epoch [3/5]
Train Loss: 1.1531
Val Loss: 1.0819
Val Accuracy: 60.17%
--------------------------------------------------


 80%|████████  | 4/5 [01:23<00:20, 20.92s/it]

Epoch [4/5]
Train Loss: 1.1208
Val Loss: 1.0580
Val Accuracy: 61.49%
--------------------------------------------------


100%|██████████| 5/5 [01:44<00:00, 20.97s/it]

Epoch [5/5]
Train Loss: 1.1081
Val Loss: 1.0603
Val Accuracy: 61.59%
--------------------------------------------------


## CIFAR-100

In [ ]:
model, transform = get_effnetb0(out_features=100)
train_dataloader, test_dataloader = get_cifar100_dataloaders(transform=transform)
train_model(model, train_dataloader, test_dataloader)

Files already downloaded and verified


 20%|██        | 1/5 [02:30<10:02, 150.74s/it]

Epoch [1/5]
Train Loss: 2.5018
Val Loss: 1.7005
Val Accuracy: 56.99%
--------------------------------------------------


 40%|████      | 2/5 [04:59<07:28, 149.56s/it]

Epoch [2/5]
Train Loss: 1.6876
Val Loss: 1.5221
Val Accuracy: 59.64%
--------------------------------------------------


...

Epoch [3/5]
Train Loss: 1.5186
Val Loss: 1.4709
Val Accuracy: 60.59%
--------------------------------------------------


 80%|████████  | 4/5 [09:59<02:30, 150.11s/it]

Epoch [4/5]
Train Loss: 1.4165
Val Loss: 1.4164
Val Accuracy: 61.65%
--------------------------------------------------


100%|██████████| 5/5 [12:29<00:00, 149.86s/it]

Epoch [5/5]
Train Loss: 1.3563
Val Loss: 1.4225
Val Accuracy: 61.39%
--------------------------------------------------


In [ ]:
model, transform = get_mobilenet()
train_dataloader, test_dataloader = get_cifar100_dataloaders(transform=transform)
train_model(model, train_dataloader, test_dataloader)

Files already downloaded and verified


 20%|██        | 1/5 [02:07<08:28, 127.16s/it]

Epoch [1/5]
Train Loss: 1.9814
Val Loss: 1.7155
Val Accuracy: 53.50%
--------------------------------------------------


 40%|████      | 2/5 [04:14<06:21, 127.15s/it]

Epoch [2/5]
Train Loss: 1.4239
Val Loss: 1.4226
Val Accuracy: 59.85%
--------------------------------------------------


...

Epoch [3/5]
Train Loss: 1.2488
Val Loss: 1.3940
Val Accuracy: 61.01%
--------------------------------------------------


 80%|████████  | 4/5 [08:29<02:07, 127.29s/it]

Epoch [4/5]
Train Loss: 1.1174
Val Loss: 1.3886
Val Accuracy: 61.15%
--------------------------------------------------


100%|██████████| 5/5 [10:35<00:00, 127.03s/it]

Epoch [5/5]
Train Loss: 1.0071
Val Loss: 1.3802
Val Accuracy: 62.25%
--------------------------------------------------


In [ ]:
model, transform = get_effnetb7(out_features=100)
train_dataloader, test_dataloader = get_cifar100_dataloaders(transform=transform)
train_model(model, train_dataloader, test_dataloader)

Files already downloaded and verified


  0%|          | 0/5 [00:00<?, ?it/s]